<h2>06. Nihai Üretim Hattı (Final Production Pipeline)</h2>

<hr>

<h3>Proje Detayları</h3>
<ul>
    <li><b>Proje:</b> FreshCart Müşteri Kaybı Tahmini (Customer Churn Prediction)</li>
    <li><b>Amaç:</b> Uçtan Uca Veri İşleme ve Model Eğitimi Hattı (End-to-End Data Processing & Model Training Pipeline)</li>
</ul>

<hr>

<h3>Amaç</h3>
<p>
    Bu betik, önceki tüm adımları (Veri Yükleme, Özellik Mühendisliği ve Modelleme) tek, tekrarlanabilir bir <b>üretim hattında</b> (pipeline) birleştirir. Bir üretim eğitimi çalıştırmasını simüle eder:
</p>

<ol>
    <li><b>Ham Veriyi Yükle</b> (Load Raw Data)</li>
    <li><b>Kesme Stratejisi Uygula</b> (Sızıntıyı Önle) (Apply Cutoff Strategy - Prevent Leakage)</li>
    <li><b>Tüm Özellikleri Oluştur</b> (RFM + Davranışsal + Gelişmiş) (Generate All Features)</li>
    <li><b>Nihai Modeli Eğit</b> (Optimize Edilmiş Hiperparametreleri Kullanarak) (Train Final Model)</li>
    <li><b>Dağıtım İçin Yapıtları Dışa Aktar</b> (Model ve Meta Veriler) (Export Artifacts)</li>
</ol>

<hr>

In [1]:
#Kütüphaneleri import etme
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib
import json
import sys
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report, precision_recall_curve
from scipy import stats

print("Kütüphaneler başarıyla yüklendi.")

Kütüphaneler başarıyla yüklendi.


In [2]:
# Özel modülleri içe aktarmak için 'src' dizinini yola ekle
sys.path.append('../src')

In [3]:
from config import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR, RANDOM_STATE
from data.data_loader import InstacartDataLoader

In [4]:
# Özellik Mühendisliği Modüllerini İçe Aktar
from features.rfm_features import RFMFeatureEngineer
from features.behavioral_features import BehavioralFeatureEngineer

print("Ortam Kurulumu Tamamlandı")

Ortam Kurulumu Tamamlandı


<h4><b>
Adım 1: Ham Veriyi Al
<h4><b>

In [5]:
def load_raw_data(self):
        """Ham Instacart veri kümelerini yükler ve önceki/eğitim siparişlerini birleştirir."""
                
        print("Ham Veri Yükleniyor...")
        
        # config.py dosyasındaki yolu kullanarak veri yükleyiciyi başlat
        loader = InstacartDataLoader(RAW_DATA_DIR)
        
        # Tüm veri kümelerini bir sözlüğe (dictionary) yükle
        data = loader.load_all_data()

        # Ana veri çerçevelerini (dataframes) ayır
        orders_df = data['orders']
        products_df = data['products']
        
        # Önceki (prior) ve eğitim (train) sipariş ürünlerini tek bir veri çerçevesinde birleştir (concatenate)
        order_products = pd.concat([
            data['order_products_prior'],
            data['order_products_train']
        ], ignore_index=True)

        print(f"Veri Yüklendi. Siparişler: {len(orders_df):,}, Ürünler: {len(products_df):,}")
        
        # Üretim hattı (pipeline) için gerekli tüm veri çerçevelerini döndür
        return orders_df, products_df, order_products

<h4><b>
Adım 2: Özellik Mühendisliği Hattı ("Sızıntısız" Mantık)
<h4><b>

In [6]:
def calculate_trend(series):
    """
    Hesaplama maliyetinden tasarruf etmek için sadece en az 2 veri noktasına sahip seriler için eğimi (slope) hesaplar.
    """
    if len(series) < 2:
        return 0
    try:
        # x zamandır (sipariş sırası), y değerdir (sepet büyüklüğü vb.)
        # stats.linregress, doğrusal regresyon parametrelerini döndürür.
        slope, _, _, _, _ = stats.linregress(np.arange(len(series)), series.values)
        return slope
    except:
        return 0

In [7]:
def run_feature_pipeline(orders_df, order_products, products_df):
    """
    Tüm özellik mühendisliği hattını 03_feature_engineering.ipynb mantığına uygun olarak çalıştırır.
    """
    print("\nÖzellik Üretim Hattı Başlatılıyor...")
    
    # 1. SIRALAMA VE BÖLME (Kesme Stratejisi)
    print("1. Kesme Stratejisi Uygulanıyor (Geçmiş ve Gelecek Ayrımı Yapılıyor)...")
    orders_sorted = orders_df.sort_values(['user_id', 'order_number'])
    last_orders = orders_sorted.groupby('user_id').tail(1)  # Hedef (Target)
    orders_history = orders_sorted.drop(last_orders.index)  # Özellikler Geçmişi (Features History)
    
    # order_products verilerini sadece geçmiş siparişler için filtrele
    op_history = order_products[order_products['order_id'].isin(orders_history['order_id'])]
    
    # 2. HEDEFLERİ OLUŞTUR
    print("2. Hedefler Oluşturuluyor...")
    labels = last_orders[['user_id', 'days_since_prior_order']].copy()
    # Eğer son siparişten bu yana geçen gün sayısı >= 30 ise 'churn' (kayıp) olarak etiketle
    labels['is_churn'] = (labels['days_since_prior_order'] >= 30).astype(int)
    
    # 3. RFM ÖZELLİKLERİ
    print("3. RFM Özellikleri Oluşturuluyor (Sadece Ham Metrikler)...")
    rfm_eng = RFMFeatureEngineer()
    rfm_feats = rfm_eng.create_all_rfm_features(orders_history, op_history)
        
    # Özel Risk ve Değer Metrikleri
    rfm_feats['clv_proxy'] = rfm_feats['total_orders'] * rfm_feats['avg_basket_size']
    rfm_feats['engagement_score'] = rfm_feats['orders_per_day'] * rfm_feats['total_items_ordered']
    rfm_feats['at_risk_score'] = rfm_feats['days_since_last_order'] / (rfm_feats['avg_days_between_orders'] + 1)
    
    # 4. DAVRANIŞSAL ÖZELLİKLER
    print("4. Davranışsal Özellikler Oluşturuluyor...")
    beh_eng = BehavioralFeatureEngineer()
    beh_feats = beh_eng.create_all_behavioral_features(orders_history, op_history, products_df)
    
    # 5. ZAMAN SERİSİ / EĞİLİM (TREND) ÖZELLİKLERİ
    print("5. Zaman Serisi Eğilimleri Türetiliyor...")
    
    # Veriyi kullanıcıya göre grupla
    # Sadece hesaplama için gerekli sütunları al
    user_trends = orders_history.groupby('user_id').agg({
        'days_since_prior_order': list  # Her kullanıcı için siparişler arasındaki günlerin listesi
    }).reset_index()
    
    # Sepet büyüklüğü eğilimi için order_products, orders ile birleşmelidir
    order_sizes = op_history.groupby('order_id').size().reset_index(name='basket_size')
    order_sizes = order_sizes.merge(orders_history[['order_id', 'user_id']], on='order_id')
    basket_trends = order_sizes.groupby('user_id').agg({'basket_size': list}).reset_index()
    
    # Eğilim Fonksiyonlarını Uygula
    user_trends['order_frequency_trend'] = user_trends['days_since_prior_order'].apply(lambda x: calculate_trend(pd.Series(x).dropna()))
    basket_trends['basket_size_trend'] = basket_trends['basket_size'].apply(lambda x: calculate_trend(pd.Series(x)))
    
    # 6. ORAN (RATIO) ÖZELLİKLERİ
    print("6. Hız (Velocity) ve İvme (Acceleration) Metrikleri Oluşturuluyor...")
    # Hız (Velocity)
    rfm_feats['purchase_velocity'] = 1 / (rfm_feats['avg_days_between_orders'] + 1)
    
    # İvme (Acceleration) (Son sipariş / Ortalama siparişler arası günler)
    # > 1 yavaşlama demektir (Kaybetme riski), < 1 hızlanma demektir
    rfm_feats['recency_acceleration'] = rfm_feats['days_since_last_order'] / (rfm_feats['avg_days_between_orders'] + 0.01)
    
    # Etkileşim (Interaction)
    rfm_feats['recency_x_frequency'] = rfm_feats['days_since_last_order'] * rfm_feats['total_orders']

    # 7. HEPSİNİ BİRLEŞTİR (MERGE)
    print("7. Özellik Kümeleri Birleştiriliyor...")
    final_df = labels[['user_id', 'is_churn']].merge(rfm_feats, on='user_id', how='left')
    final_df = final_df.merge(beh_feats, on='user_id', how='left')
    final_df = final_df.merge(user_trends[['user_id', 'order_frequency_trend']], on='user_id', how='left')
    final_df = final_df.merge(basket_trends[['user_id', 'basket_size_trend']], on='user_id', how='left')
    
    # NaN değerleri 0 ile doldur
    final_df = final_df.fillna(0)
    
    return final_df

In [8]:
# Eğitim (TRAINING) verisinden bölmeleri (bins) almak için yardımcı fonksiyon
def get_qcut_bins(data_series, q):
    """YALNIZCA eğitim veri serisinden kantil bölmelerini (quantile bins) hesaplar."""
    # retbins=True bölme kenarlarını (bin edges) döndürür
    return pd.qcut(data_series, q=q, retbins=True, duplicates='drop')[1]

# Bölmeleri TEST verisine uygulamak için yardımcı fonksiyon
def apply_qcut_bins(test_series, bins, labels):
    """Önceden hesaplanmış bölmeleri eğitim verisinden test serisine uygular."""
    # pd.cut, eğitim kümesinden hesaplanan açık bölmeleri uygulamak için kullanılır.
    # right=False aralığın [a, b) olmasını sağlar, bu da veri sızıntısını (leakage) önler.
    return pd.cut(
        test_series, 
        bins=bins, 
        labels=labels, 
        include_lowest=True, 
        right=False  # Tutarlılık ve sızıntıyı önlemek için kritik
    ).astype(float).fillna(0).astype(int) # Test verisi eğitim bölmelerinin dışına düşerse dayanıklılık için fillna(0)

<h4><b>
Adım 3: Nihai Model Eğitimi (Final Model Training)
<h4><b>

In [9]:
def train_model(final_dataset):
    print("\nEğitim İçin Veri Hazırlanıyor...")

    # 1. X ve y'yi Hazırla
    # 'user_id' ve 'is_churn' sütunlarını özelliklerden kaldır
    feature_cols = [c for c in final_dataset.columns if c not in ['user_id', 'is_churn']]
    
    X = final_dataset[feature_cols].copy()
    y = final_dataset['is_churn']

    # 2. Eğitim/Doğrulama İçin Bölme (Split)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )
    print(f"Veri bölme tamamlandı. X_train boyutu: {X_train.shape}, X_test boyutu: {X_test.shape}")

    # 3. En İyi Hiperparametreleri Yükle
    try:
        with open(MODEL_DIR / 'best_params.json', 'r') as f:
            best_params = json.load(f)
        print("En İyi Hiperparametreler önceki adımdan yüklendi.")
    except Exception as e:
        print(f"En iyi parametreler dosyası bulunamadı. Varsayılan parametreler kullanılıyor. Hata: {e}")
        best_params = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'learning_rate': 0.05,
            'n_estimators': 1000,
            'scale_pos_weight': 2.3
        }

    # 4. LightGBM'i Eğit
    print("\nNihai LightGBM Modeli Eğitiliyor...")
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_test, label=y_test, reference=dtrain)

    final_model = lgb.train(
        best_params,
        dtrain,
        valid_sets=[dvalid],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )
    
    # --- OPTİMAL EŞİK DEĞERİ (THRESHOLD) HESAPLAMASI ---
    print("\nOptimal Eşik Değeri Hesaplanıyor...")
    
    y_pred_proba = final_model.predict(X_test)
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba)
    # F1 skorunu hesapla
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    
    print(f"Bulunan En İyi Eşik Değeri: {best_threshold:.4f}")
    
    # Eşik değerinin modelle eşleşmesini sağlamak için hemen kaydet
    threshold_path = MODEL_DIR / 'optimal_threshold.json'
    with open(threshold_path, 'w') as f:
        json.dump({'threshold': float(best_threshold)}, f)
    print(f"Eşik Değeri şuraya kaydedildi: {threshold_path}")

    print("Model Eğitimi Tamamlandı.")
    
    # feature_cols listesini ham haliyle döndürüyoruz (Skorlama yok)
    return final_model, X_test, y_test, feature_cols, best_threshold

<h4><b>
Adım 4: Hızlı Doğrulama ve Sağlama Kontrolü (Quick Validation & Sanity Check)
<h4><b>

In [10]:
def validate_model(model, X_test, y_test, threshold):
    print("\nModel Performansı Doğrulanıyor...")
    
    y_pred_prob = model.predict(X_test)
    
    # Dinamik eşik değeri (threshold) kullanıyoruz
    y_pred = (y_pred_prob >= threshold).astype(int)

    # AUC (Eğri Altındaki Alan) skorunu hesapla
    auc = roc_auc_score(y_test, y_pred_prob)
    # F1 skorunu hesapla
    f1 = f1_score(y_test, y_pred)
    
    print(f"Nihai AUC Skoru: {auc:.4f}")
    print(f"Nihai F1 Skoru : {f1:.4f} (eşik değerinde {threshold:.4f})")
    
    print("\nSınıflandırma Raporu (Classification Report):")
    print(classification_report(y_test, y_pred))

<h4><b>
Adım 5: Dağıtım İçin Ürünleri Dışa Aktar (Export Artifacts for Deployment)
<h4><b>

In [11]:
def save_artifacts(model, feature_cols, final_dataset):
    print("\nÜretim Yapıtları Dışa Aktarılıyor...")

    # 1. Modeli Kaydet
    model_path = MODEL_DIR / 'final_model_optimized.pkl'
    joblib.dump(model, model_path)
    print(f" -> Model şuraya kaydedildi: {model_path}")

    # 2. Özellik Listesini Kaydet (API girdi doğrulaması için kritiktir)
    # PROCESSED dizinine kaydet (Veri Hattı Standardı)
    feature_path_processed = PROCESSED_DATA_DIR / 'model_features.json'
    with open(feature_path_processed, 'w') as f:
        json.dump(feature_cols, f)
    print(f" -> Özellik listesi şuraya kaydedildi: {feature_path_processed}")

    # MODELS dizinine kaydet (App.py Standardı - KEYERROR hatası için düzeltme)
    feature_path_models = MODEL_DIR / 'feature_names.json'
    with open(feature_path_models, 'w') as f:
        json.dump(feature_cols, f)
    print(f" -> Özellik listesi şuraya senkronize edildi: {feature_path_models}")

    # 3. Veri Kümesini Kaydet (İsteğe Bağlı, Dashboard EDA için)
    data_path = PROCESSED_DATA_DIR / 'final_features_advanced.parquet'
    final_dataset.to_parquet(data_path)
    print(f" -> İşlenmiş veri şuraya kaydedildi: {data_path}")

    print("\nÜRETİM HATTI BAŞARIYLA TAMAMLANDI!")

<h4><b>
ANA ÇALIŞTIRMA (MAIN EXECUTION)
<h4><b>

In [12]:
if __name__ == "__main__":
    # 1. Veriyi Yükle
    data_loader = InstacartDataLoader(RAW_DATA_DIR) 
    data_dict = data_loader.load_all_data()
    print("Ham Veri Yüklendi.")
    
    orders_df = data_dict['orders']
    products_df = data_dict['products']

    # Önceki ve eğitim siparişlerini birleştir
    order_products = pd.concat([
        data_dict['order_products_prior'],
        data_dict['order_products_train']
    ])
    
    # 2. Özellik Üretim Hattını Çalıştır
    final_dataset = run_feature_pipeline(orders_df, order_products, products_df)
    print(f"\nÜretim Hattı Tamamlandı. Veri Kümesi Boyutu (Shape): {final_dataset.shape}")
    
    # 3. Modeli Eğit (Ayrıca Eşik Değerini de alıyoruz)
    final_model, X_test, y_test, feature_cols, best_threshold = train_model(final_dataset)
    
    # 4. Doğrula (Eşik Değerini sağlıyoruz)
    validate_model(final_model, X_test, y_test, best_threshold)
    
    # 5. Kaydet
    save_artifacts(final_model, feature_cols, final_dataset)

INFO:data.data_loader:Instacart veri setleri yükleniyor...
INFO:data.data_loader:Yükleniyor: orders.csv...
INFO:data.data_loader:Yüklendi orders: (3421083, 7)
INFO:data.data_loader:Yükleniyor: order_products__prior.csv...
INFO:data.data_loader:Yüklendi order_products_prior: (32434489, 4)
INFO:data.data_loader:Yükleniyor: order_products__train.csv...
INFO:data.data_loader:Yüklendi order_products_train: (1384617, 4)
INFO:data.data_loader:Yükleniyor: products.csv...
INFO:data.data_loader:Yüklendi products: (49688, 4)
INFO:data.data_loader:Yükleniyor: aisles.csv...
INFO:data.data_loader:Yüklendi aisles: (134, 2)
INFO:data.data_loader:Yükleniyor: departments.csv...
INFO:data.data_loader:Yüklendi departments: (21, 2)
INFO:data.data_loader:Tüm veri setleri başarıyla yüklendi!

INFO:data.data_loader:================================================================================
INFO:data.data_loader:VERİ ÖZETİ
INFO:data.data_loader:=============================================================

Ham Veri Yüklendi.

Özellik Üretim Hattı Başlatılıyor...
1. Kesme Stratejisi Uygulanıyor (Geçmiş ve Gelecek Ayrımı Yapılıyor)...


INFO:features.rfm_features:RFM özellikleri oluşturuluyor...
INFO:features.rfm_features:Yenilik özellikleri oluşturuluyor...


2. Hedefler Oluşturuluyor...
3. RFM Özellikleri Oluşturuluyor (Sadece Ham Metrikler)...


INFO:features.rfm_features:Sıklık özellikleri oluşturuluyor...
INFO:features.rfm_features:Parasal özellikler oluşturuluyor (sepet büyüklüğünü vekil olarak kullanarak)...
INFO:features.rfm_features:14 adet RFM özelliği oluşturuldu
INFO:features.rfm_features:Özellikler: ['days_since_last_order', 'days_since_first_order', 'customer_age_days', 'avg_days_between_orders', 'total_orders', 'orders_per_day', 'order_regularity', 'std_days_between_orders', 'avg_basket_size', 'total_items_ordered', 'basket_size_std', 'basket_size_cv', 'avg_unique_products_per_order', 'total_unique_products_ordered']
INFO:features.behavioral_features:Davranışsal özellikler oluşturuluyor...
INFO:features.behavioral_features:Zaman bazlı özellikler oluşturuluyor...


4. Davranışsal Özellikler Oluşturuluyor...


d:\egitim_ve_calismalar\Lodos Makine Öğrenmesi Bootcamp 02.11.2025\html\FreshCart-Churn-Prediction\notebooks\../src\features\behavioral_features.py:104: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weekend_orders = orders_df.groupby('user_id').apply(
d:\egitim_ve_calismalar\Lodos Makine Öğrenmesi Bootcamp 02.11.2025\html\FreshCart-Churn-Prediction\notebooks\../src\features\behavioral_features.py:110: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after grou

5. Zaman Serisi Eğilimleri Türetiliyor...
6. Hız (Velocity) ve İvme (Acceleration) Metrikleri Oluşturuluyor...
7. Özellik Kümeleri Birleştiriliyor...

Üretim Hattı Tamamlandı. Veri Kümesi Boyutu (Shape): (206209, 46)

Eğitim İçin Veri Hazırlanıyor...
Veri bölme tamamlandı. X_train boyutu: (164967, 44), X_test boyutu: (41242, 44)
En İyi Hiperparametreler önceki adımdan yüklendi.

Nihai LightGBM Modeli Eğitiliyor...
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.764151
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.764151

Optimal Eşik Değeri Hesaplanıyor...
Bulunan En İyi Eşik Değeri: 0.4565
Eşik Değeri şuraya kaydedildi: d:\egitim_ve_calismalar\Lodos Makine Öğrenmesi Bootcamp 02.11.2025\html\FreshCart-Churn-Prediction\notebooks\..\models\optimal_threshold.json
Model Eğitimi Tamamlandı.

Model Performansı Doğrulanıyor...
Nihai AUC Skoru: 0.7642
Nihai F1 Skoru : 0.5893 (eşik değerinde 0.4565)

Sınıflandırma Raporu (Classificati